# Computational Social Science Project #2 
Group members: Pei-Ming (Vincent) Chen, Lawrence Liu, & Kelly Quinn 

Date: 10/20/2020

# 1. Set Up

In [ ]:
## import necessary packages
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
from sklearn.preprocessing import OneHotEncoder, scale
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import rc
from matplotlib.ticker import FormatStrFormatter
import plotly.graph_objects as go
import nbformat
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, ElasticNetCV
from sklearn.metrics import mean_squared_error, make_scorer, r2_score
import seaborn as sns
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.exceptions import ConvergenceWarning
from sklearn.ensemble import RandomForestRegressor
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import RFE
from sklearn.metrics import r2_score
import math
from sklearn.model_selection import cross_val_score


# use random seed for consistent results 
import random
random.seed(273)

# 2. Exploratory Data Analysis

In [ ]:
##Cleaning for Exploratory Data Analysis

diabetes_filename = 'Diabetes with Population Info by County 2017.csv'
df = pd.read_csv(diabetes_filename, dtype={"CountyFIPS": str}) #CountyFips needs to be a string so leading 0 isn't dropped

df = df[df.Diabetes_Number != "Suppressed"] #drop row with "Suppressed" value in Diabetes_Number
df["Diabetes_Number"] = df["Diabetes_Number"].astype('int64') #convert Diabetes Number to int
df['Rate_Per_Thousand'] = df['Diabetes_Number']*1000

#df["Diabetes_Number_PC"] = df["Diabetes_Number"] / df["race_total population"] #create diabetes number per capita column

df = df[df["Obesity_Number"] != 'No Data']
df = df[df['Physical_Inactivity_Number'] != "No Data"]

df = df.dropna()

target_col = 'sex and age_total population_65 years and over_sex ratio (males per 100 females)'
df = df[df[target_col] != '-']

##Remove duplicate columns

## Identify more duplicate columns - source: https://thispointer.com/how-to-find-drop-duplicate-columns-in-a-dataframe-python-pandas/ - kq
def getDuplicateColumns(df):
    '''
    Get a list of duplicate columns.
    It will iterate over all the columns in dataframe and find the columns whose contents are duplicate.
    :param df: Dataframe object
    :return: List of columns whose contents are duplicates.
    '''
    duplicateColumnNames = set()
    # Iterate over all the columns in dataframe
    for x in range(df.shape[1]):
        # Select column at xth index.
        col = df.iloc[:, x]
        # Iterate over all the columns in DataFrame from (x+1)th index till end
        for y in range(x + 1, df.shape[1]):
            # Select column at yth index.
            otherCol = df.iloc[:, y]
            # Check if two columns at x 7 y index are equal
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
    return list(duplicateColumnNames)

duplicateColumnNames = getDuplicateColumns(df)
print('Duplicate Columns are as follows')
for col in duplicateColumnNames:
    print('Column name : ', col)

#drop all duplicated columns
df = df.drop(columns=['race_total population_one race_1', 'race_total population_two or more races_1',
                     'sex and age_total population_18 years and over_1', 
                      'sex and age_total population_65 years and over_1',
                      'sex and age_total population', 
                      'race alone or in combination with one or more other races_total population',
                      'hispanic or latino and race_total population'
                     ]) 

#drop ratio + median columns
df = df.drop(columns = ['sex and age_total population_65 years and over_sex ratio (males per 100 females)', 
                       'sex and age_total population_18 years and over_sex ratio (males per 100 females)', 
                       'sex and age_total population_median age (years)']) 

df.shape # 3112*86

First, we convert existing counts to percentages to calculate rates:

In [ ]:
# select count variables to rc to percentages
rc_cols = [col for col in df.columns if col not in ['County', 'State', 'CountyFIPS']]
           
           
df[rc_cols] = df[rc_cols].apply(pd.to_numeric, errors='coerce') # recode to numeric

# divide all columns but those listed above by total population 
df[rc_cols] = df[rc_cols].div(df['race_total population'], axis=0)

In [ ]:
# check work: all rates are bounded by 0 and 1 as expected
pd.set_option('display.max_columns', None)
df_summary = df.describe().transpose()

with pd.option_context('display.max_rows', 100, 'display.max_columns', None): 
    display(df_summary.iloc[ : ,[0,1,3,7]])

## (i) Choropleth Map: Diabates Number by County

In [ ]:
# Documentation relied upon at: https://plotly.com/python/choropleth-maps/

from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
    
from plotly import express as px

df_choro = df[['CountyFIPS', 'Diabetes_Number']] #make dataframe with two columns to speed up process

fig = px.choropleth(df_choro, geojson=counties, locations='CountyFIPS', color='Diabetes_Number',
                           color_continuous_scale="Viridis", 
                           scope="usa",
                           labels={'Diabetes_Number':'Diabetes Rate'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

The choropleth map illustrates how diabetes rates are distributed across the country. We notice that  diabetes rates in some counties are above 20% of the county population. We also see that the counties with the highest diabetes rates tend to be concentrated in the Southeastern United States.

## (ii) Diabetes Rates by Subgroup Distribution

### (iia) Diabetes Rate by County Age Breakdown 

In [ ]:
rc('font', weight='bold')

bar_u25 = df['sex and age_total population_under 5 years']*100 + df['sex and age_total population_5 to 9 years']*100 + df['sex and age_total population_10 to 14 years']*100 + df['sex and age_total population_15 to 19 years'] * 100 + df['sex and age_total population_20 to 24 years']*100 
bar_2544 = df['sex and age_total population_25 to 34 years']*100 + df['sex and age_total population_35 to 44 years']*100
bar_4559 = df['sex and age_total population_45 to 54 years']*100 + df['sex and age_total population_55 to 59 years']*100 
bar_6075 = df['sex and age_total population_60 to 64 years'] + df['sex and age_total population_65 to 74 years']*100 
bar_o75 = df['sex and age_total population_75 to 84 years']*100 + df['sex and age_total population_85 years and over']*100 

# cumulative height of bars
barx1 = np.add(bar_u25,bar_2544).tolist() # first two bars
barx2 = np.add(barx1,bar_4559).tolist() # first three bars
barx3 = np.add(barx2,bar_6075).tolist() # first four bars

p1 = plt.bar(df['Rate_Per_Thousand'],bar_u25, color='maroon')
p2 = plt.bar(df['Rate_Per_Thousand'],bar_2544, bottom=bar_u25, color='salmon')
p3 = plt.bar(df['Rate_Per_Thousand'],bar_4559, bottom=barx1, color='lightcoral')
p4 = plt.bar(df['Rate_Per_Thousand'],bar_6075, bottom=barx2, color='mistyrose')
p5 = plt.bar(df['Rate_Per_Thousand'],bar_o75, bottom=barx3, color='seashell')


plt.xlabel("# with Diabetes Per Thousand People")
plt.ylabel("Age Distribution")
plt.legend((p1[0],p2[0],p3[0],p4[4],p5[0]), ("<20 yo", "25-44 yo", "45-59 yo", "60-75", '75+ yo'),loc="upper left")

plt.show()

### (iib) Diabetes Rate by Race Breakdown 

In [ ]:
bar_amin = df['race_total population_one race_american indian and alaska native'] * 100 
bar_asian = df['race_total population_one race_asian'] *100
bar_blk = df['race_total population_one race_black or african american'] * 100 
bar_pacisl = df['race_total population_one race_native hawaiian and other pacific islander'] *100
bar_other = df['race_total population_one race_some other race'] *100 + df['race_total population_two or more races'] * 100 
bar_white = df['race_total population_one race_white'] *100

# cumulative height of bars 
barx1 = np.add(bar_amin,bar_asian).tolist() 
barx2 = np.add(barx1,bar_blk).tolist()
barx3 = np.add(barx2,bar_pacisl).tolist() 
barx4 = np.add(barx3,bar_other).tolist() 

p1 = plt.bar(df['Rate_Per_Thousand'],bar_amin, color='darkblue')
p2 = plt.bar(df['Rate_Per_Thousand'],bar_asian, bottom=bar_amin, color='royalblue')
p3 = plt.bar(df['Rate_Per_Thousand'],bar_blk, bottom=barx1, color='dodgerblue')
p4 = plt.bar(df['Rate_Per_Thousand'],bar_pacisl, bottom=barx2, color='lightskyblue')
p5 = plt.bar(df['Rate_Per_Thousand'],bar_other, bottom=barx3, color='powderblue')
p6 = plt.bar(df['Rate_Per_Thousand'],bar_white, bottom=barx4, color='lightcyan')


plt.xlabel("# with Diabetes Per Thousand People")
plt.ylabel("Race Distribution")
plt.legend((p1[0],p2[0],p3[0],p4[4],p5[0],p6[0]), ("Am. Indian", "Asian", "Black", "Pac. Islander", "Other", "White"),loc="upper left")

plt.show()

## INSERT EXPLANATION OF SUBGROUP DISTRIBUTIONS HERE. 

## (iii) Correlation Plot

In [ ]:
### EDA: get correlation with Diabetes_Number
topn = 5
corr_w_diab_num = df.corr()['Diabetes_Number']
cols = corr_w_diab_num.axes[0].to_list()
col2corr = {col:corr_w_diab_num[col] for col in cols if col not in ['Rate_Per_Thousand', 'race_total population']}
col_corr = sorted(col2corr.items(), key=lambda x: abs(x[1]), reverse=True)
top_correlated_cols = [p[0] for p in col_corr[:topn]]

# add newline in place of underscore for labels
df_correlation_plot = df[top_correlated_cols]
renamed_col = {col: '\n'.join(col.split('_')) for col in top_correlated_cols}
df_correlation_plot = df_correlation_plot.rename(columns=renamed_col)

axes = scatter_matrix(df_correlation_plot, figsize=(30, 30)); # adding a semi colon hides the code - kq
for i in range(topn):
    for j in range(topn):
        # set the format to two decimal numbers
        axes[i, j].yaxis.set_major_formatter(FormatStrFormatter('%.2f'))

mpl.rcParams['axes.labelsize'] = 25


In the correlation plot, we plot the correlation between top five features with highest absolute correlation with diabetes rate. Note that all features are now normalized by the population. We found that all of the features have medium to low correlation with diabetes rate: the feature most correlated with diabetes rate is the percentage of African American, with a correlation coefficient of .37. The rest of the features have absolute correlation coefficients lower than .3. The correlation plot also shows that features among the same category, e.g. racial features or population-related features, have high correlations while features from different categories tend to have low correlation. This suggests that when conducting feature selection, we can pick one feature from each category to get maximally uncorrelated set of features.

# 3. Preparing to Fit Models

### 3.1. Clean Data

In [ ]:
##Feature Selection

#Convert state to dummy variables
onehotencoder = OneHotEncoder()
onehotencoder.fit(df['State'].to_numpy().reshape(-1, 1))
state_array = onehotencoder.transform(df['State'].to_numpy().reshape(-1, 1)).todense()

df_states = pd.DataFrame(state_array)
df_wo_states = df.drop(columns=['State'])
df_cleaned = pd.concat([df_states, df_wo_states], axis=1)

#Drop County and FIPS
df_cleaned.drop(columns=['County', 'CountyFIPS'])

#Drop rate per thosuand feature created for EDA.
df_cleaned = df.drop(columns = "Rate_Per_Thousand")

### 3.3 Feature Selection
Note: we do feature selection before partitioning the data.

We are interested in only keeping the features that are highly correlated with our target of interest: diabetes rate. Features that are not highly correlated with the target do not provide enough information to improve our prediction, and keeping such features would hinder interpretability and be computationally expensive.

In [ ]:
#Select variable that have correlations above 0.3 with Diabetes_Number
cor = df_cleaned.corr()
cor_target = abs(cor["Diabetes_Number"])

relevant_features = cor_target[cor_target>0.3]
relevant_features

In [ ]:
#Recreate data frame, only keeping features that are highly correalted with target

df_cleaned = df_cleaned.filter(['Diabetes_Number', 'Obesity_Number', 'Physical_Inactivity_Number',
                                        'race_total population_one race_black or african american', 
                                        'race alone or in combination with one or more other races_total population_black or african american',
                                        'hispanic or latino and race_total population_not hispanic or latino_black or african american alone'])


We then computed the linear correlation between the remaining features. 

In [ ]:
plt.figure(figsize=(12,10))
cor = df_cleaned.corr()
sns.heatmap(cor, annot=True, cmap=plt.cm.Reds)
plt.show()

Since 'race_total population_one race_black or african american', 'race alone or in combination with one or more other races_total population_black or african american', and 'hispanic or latino and race_total population_not hispanic or latino_black or african american alone' are perfectly correlated with each other (corr = 1), we select the feature that has the strongest correlation with the target: 'hispanic or latino and race_total population_not hispanic or latino_black or african american' alone.

In [ ]:
df_cleaned = df_cleaned.drop(columns = ['race_total population_one race_black or african american',
                               'race alone or in combination with one or more other races_total population_black or african american'])

In [ ]:
#Convert observations to floats
for col in df_cleaned.columns:
    df_cleaned[col] = df_cleaned[col].astype('float64')

print("Shape of Cleaned DataFrame:", df.shape)
df_cleaned.head()

### 3.2. Partition Data
The percentages of training set, validation set, and test set are 70%, 20%, and 10% respectively. The training set is used to train the models. The validation set is used for tuning hyperparameters and model selection. The test set is used to test the accuracy of the final model. A high training set percentage will lead to lower bias of the models since more data are used in training, but at the same time this will decrease the amount of data for computing accuracy, resulting in higher variance in validation and testing accuracy. We chose a high percentage for the training set since we want to reserve most of the data for model training.

In [ ]:
##Splittng into features and target
features = df_cleaned.drop(columns = ["Diabetes_Number"])
target = df_cleaned.filter(["Diabetes_Number"])

## Data partition
n_data = df_cleaned.shape[0]
n_train = int(n_data*0.7)
n_valid = int(n_data*0.2)
n_test = n_data - n_train - n_valid

labels = np.repeat(['train', 'valid', 'test'], (n_train, n_valid, n_test))
np.random.seed(0)
randomized_labels = np.random.choice(labels, n_data, replace=False)
df_cleaned['group'] = randomized_labels

df_train = df_cleaned[df_cleaned['group'] == 'train']
df_valid = df_cleaned[df_cleaned['group'] == 'valid']
df_test = df_cleaned[df_cleaned['group'] == 'test']

df_train = df_train.drop(columns=['group'])
df_valid = df_valid.drop(columns=['group'])
df_test = df_test.drop(columns=['group'])

y_train = df_train['Diabetes_Number']
X_train = df_train.drop(columns=['Diabetes_Number'])
y_valid = df_valid['Diabetes_Number']
X_valid = df_valid.drop(columns=['Diabetes_Number'])
X_test = df_test.drop(columns=['Diabetes_Number'])
y_test = df_test['Diabetes_Number']

#standardize features
X_train_scaled = scale(X_train)
X_valid_scaled = scale(X_valid)
X_test_scaled = scale(X_test)
features_scaled = scale(features)

# 4. Train Models, Predict on Validation Set, Refine Models
The five models that we picked are OLS regression, ridge regression, LASSO regression, Elastic-Net, and Random Forest. Hyperparameters are chosen using grid search on the training set except for LASSO, where we had to manually set an extremely small alpha to prevent shrinking all coefficients to zero.

## (i) OLS Regression Model
OLS model captures the linear relationship between IVs and DV. The strength of OLS is the relative ease of interpretting the coefficients, which quantifies the magnitude and direction of linear effects of IVs on the DV. The drawback of OLS is the lower prediction accuracy compared to other machine learning methods. In addition, if there are highly correlated features in the dataset, the coefficients could be poorly determined, leading to extremely postive or negative coefficients. Interpretation of coefficients could also be a problem if there are a large number of features, since most features will have non-zero coefficients in OLS. Given its simplicity, OLS is a good start for most regression problems.

In [ ]:
#Determine hyperparameters to use
param_grid = {'fit_intercept': ['True', 'False'],
              'normalize': ['True', 'False']}

lin_grid_reg = GridSearchCV(LinearRegression(), param_grid, cv=3, iid=False)
lin_grid_reg.fit(X_train_scaled, y_train)

best_index = np.argmax(lin_grid_reg.cv_results_["mean_test_score"])

print(lin_grid_reg.cv_results_["params"][best_index])

In [ ]:
#Fit linear regression model on  training data using best hyperparameters
linear_reg = LinearRegression(**lin_grid_reg.cv_results_["params"][best_index]).fit(X_train_scaled, y_train)


In [ ]:
# Evaluate how well model fits the training data
lin_y_hat_train = linear_reg.predict(X_train_scaled)
lin_r2_score =  r2_score(y_train, lin_y_hat_train)
print("R^2 of OLS Training Model:", lin_r2_score)
print("Correlation of OLS Training Model:", math.sqrt(lin_r2_score))

sns.scatterplot(y_train, lin_y_hat_train)
plt.title('Linear Model (OLS) Predicted v. Actual on Training Data')
plt.xlabel('actual value')
plt.ylabel('predicted value')
plt.show()

In [ ]:
# Create a dataframe with the coefficient and feature names
lin_reg_data = pd.DataFrame([linear_reg.coef_, X_train.columns]).T
lin_reg_data.columns = ['Coefficient', 'Feature']
# Plot
ax = sns.barplot(x="Coefficient", y="Feature", data=lin_reg_data)
ax.set_title("OLS Coefficients")
plt.show()

## (ii) Ridge Regression Model
Ridge regression also aims to capture linear relations between IVs and the DV, but differ from OLS in that it imposes constraints on the squared sum of coefficients. The constraint is designed to alleviate the wildly varied coefficients when having correlated features in OLS. Similar to OLS, ridge regression also suffers from interpertability issues when having a large number of features. Since from EDA we found a few strongly correlated features in the dataset, ridge regression appears to be an apporiate choice.

In [ ]:
#Determine hyperparameters to use for ridge
param_grid = {'alpha': np.arange(.1, 1, .1),
               'normalize': ['True', 'False'],
             'fit_intercept': ['True', 'False'],
             'solver': ['auto', 'svd', 'cholesky', 'lsqr']}

ridge_grid_reg = GridSearchCV(Ridge(), param_grid, cv=3, iid=False)
ridge_grid_reg.fit(X_train_scaled, y_train)

print(ridge_grid_reg.cv_results_["params"][best_index])



In [ ]:
#Fit ridge regression model on  training data using best hyperparameters
ridge_reg = Ridge(**ridge_grid_reg.cv_results_["params"][best_index]).fit(X_train_scaled, y_train)

In [ ]:
# Evaluate how well model fits the training data
ridge_y_hat_train = ridge_reg.predict(X_train_scaled)
ridge_r2_score = r2_score(y_train, ridge_y_hat_train)
print("R^2 of Ridge Training Model:", ridge_r2_score)
print("Correlation of Ridge Training Model", math.sqrt(ridge_r2_score))

sns.scatterplot(y_train, ridge_y_hat_train)
plt.title('Ridge Predicted v. Actual on Training Data')
plt.xlabel('actual value')
plt.ylabel('predicted value')
plt.show()

In [ ]:
# Create a dataframe with the coefficient and feature names
ridge_reg_data = pd.DataFrame([ridge_reg.coef_, X_train.columns]).T
ridge_reg_data.columns = ['Coefficient', 'Feature']
# Plot
ax = sns.barplot(x="Coefficient", y="Feature", data=ridge_reg_data)
ax.set_title("Ridge Coefficients")
plt.show()

## (iii) LASSO Regression Model
LASSO is similar to ridge regression in the sense that it also places constraint on the size of the coefficients, but instead of limiting the squared sum of coefficients, LASSO limits the absolute sum of coefficients. Apart from alleviating the issue of wildly varied cofficients of OLS, the absolute sum constraint will shrink some of the coefficients to zero, leaving only the most important features with nonzero coefficients. This improves the interpratbility of coefficients. On the other hand, if the constraint parameter is too large, LASSO could shrink all coefficients to zero, so careful choice of the constraint parameter is necessary. Given the many features in this machine learning problem, LASSO serves as a reasonable model to try.

In [ ]:
##Determine hyperparameters to use
param_grid = {'normalize': ['True', 'False'],
             'fit_intercept': ['True', 'False'],
             'selection': ['cyclic', 'random'],
             }
lasso_grid_reg = GridSearchCV(Lasso(max_iter=10000), param_grid, cv=3, iid=False)
lasso_grid_reg.fit(X_train_scaled, y_train)

best_index = np.argmax(lasso_grid_reg.cv_results_["mean_test_score"])
print(lasso_grid_reg.cv_results_["params"][best_index])

In [ ]:
lasso_reg = Lasso(max_iter = 10000, alpha = .00000000001, fit_intercept = True, normalize = True, selection = 'cyclic').fit(X_train_scaled, y_train)
#need to hard-code very small alpha so coefficients appear
#lasso_reg = Lasso(**lasso_grid_reg.cv_results_["params"][best_index]).fit(X_train_scaled, y_train)


In [ ]:
# Evaluate how well model fits the training data
lasso_y_hat_train = lasso_reg.predict(X_train_scaled)
lasso_r2_score = r2_score(y_train, lasso_y_hat_train)
print("R^2 of Lasso Training Model:", lasso_r2_score)
print("Correlation of Lasso Training Model", math.sqrt(lasso_r2_score))

sns.scatterplot(y_train, lasso_y_hat_train)
plt.title('Lasso Predicted v. Actual on Training Data')
plt.xlabel('actual value')
plt.ylabel('predicted value')
plt.show()

In [ ]:
# Create a dataframe with the coefficient and feature names
lasso_reg_data = pd.DataFrame([lasso_reg.coef_, X_train.columns]).T
lasso_reg_data.columns = ['Coefficient', 'Feature']

# Plot
ax = sns.barplot(x="Coefficient", y="Feature", data=lasso_reg_data)
ax.set_title("LASSO Coefficients")
plt.show()

## (iv) Elastic-Net Model
## ADD DESCRIPTION OF ELASTIC-NET HERE

In [ ]:
#Determine hyperparameters to use
param_grid = {'alpha': np.arange(0.1,1.0,0.1),
             'l1_ratio': np.arange(0.0,1.0,.05),
             'fit_intercept': ['True', 'False'], 
             'normalize': ['True', 'False'],
             'selection': ['cyclic', 'random']}

enet = ElasticNet(max_iter=1000) 
import warnings # hide convergence warnings 
warnings.filterwarnings('ignore', category=ConvergenceWarning)
#I think it's would be better to keep the convergence warning, so that we know we have to increase max_iter when the method does not converge - VC
# ^ i agree it's safer to include them, but raising max_iter doesn't prevent the warning either and takes a WILD amount of time, so leaving for now until we figure out why - kq 
enet_grid_reg = GridSearchCV(enet, param_grid, scoring='r2', cv=3, iid=False)

enet_grid_reg.fit(X_train_scaled, y_train)

best_index = np.argmax(enet_grid_reg.cv_results_["mean_test_score"])

print(enet_grid_reg.cv_results_["params"][best_index])

In [ ]:
##Fit enet regression model on  training data using best hyperparameters
enet_reg = ElasticNet(**enet_grid_reg.cv_results_["params"][best_index]).fit(X_train_scaled, y_train)


In [ ]:
# Evaluate how well model fits the training data
enet_y_hat_train = enet_reg.predict(X_train_scaled)
enet_r2_score = r2_score(y_train, enet_y_hat_train)
print("R^2 of Elastic Net Training Model:", enet_r2_score)
print("Correlation of Elastic Net Training Model", math.sqrt(enet_r2_score))

sns.scatterplot(y_train, enet_y_hat_train)
plt.title('Elastic Nete Predicted v. Actual on Training Data')
plt.xlabel('actual value')
plt.ylabel('predicted value')
plt.show()

In [ ]:
# Create a dataframe with the coefficient and feature names
enet_reg_data = pd.DataFrame([enet_reg.coef_, X_train.columns]).T
enet_reg_data.columns = ['Coefficient', 'Feature']

# Plot
ax = sns.barplot(x="Coefficient", y="Feature", data=enet_reg_data)
ax.set_title("Elastic Net Coefficients")
plt.show()

The hyperparameter tuning of the elastic net model yielded a L1 penalty of zero, which means this model is virtually identical to the ridge regression but with different hyperparameters. This is supported by the fact that the coefficients (and, by extension, MSEs) are quite similar in the two models.

## (v) Random Forest Model
## ADD DESCRIPTION OF RANDOM FOREST HERE

In [ ]:
# Number of trees in random forest - need to tune this a bit more still and use random numbers for the # levels and samples - kq 
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 100, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [2,40]
# Minimum number of samples required to split a node
min_samples_split = [2, 20]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1,20]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_grid_reg = GridSearchCV(rf,param_grid, cv = 3, n_jobs= -1, verbose = 2)
rf_grid_reg.fit(X_train_scaled, y_train)

best_index = np.argmax(rf_grid_reg.cv_results_["mean_test_score"])

print(rf_grid_reg.cv_results_["params"][best_index])

In [ ]:
rf_reg = RandomForestRegressor(**rf_grid_reg.cv_results_["params"][best_index]).fit(X_train_scaled, y_train)

In [ ]:
importances = list(rf_reg.feature_importances_)

# Set the style
plt.style.use('fivethirtyeight')
# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical')
# Tick labels for x axis
plt.xticks(x_values, X_train.columns, rotation='vertical')
# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Feature'); plt.title('Feature Importances');

## 5. Validate and Refine Models

### 5.1. Predict on the Validation Set

Having fitted each model to the training data, we now predict on the validation set. 

In [ ]:
#Predict each fitted model on the validation set

linear_y_hat = linear_reg.predict(X_valid_scaled)
ridge_y_hat = ridge_reg.predict(X_valid_scaled)
lasso_y_hat = lasso_reg.predict(X_valid_scaled)
enet_y_hat = enet_reg.predict(X_valid_scaled)
rf_y_hat = rf_reg.predict(X_valid_scaled)

We then compare the strength of the models by comparing the mean squared error of each model. We will select the model with the lowest mean squared error and predict that model on the test set.

In [ ]:
#Calcualte and compare MSEs 

mse_linear = mean_squared_error(y_valid, linear_y_hat)
mse_ridge = mean_squared_error(y_valid, ridge_y_hat)
mse_lasso = mean_squared_error(y_valid, lasso_y_hat) 
mse_enet = mean_squared_error(y_valid, enet_y_hat)
mse_rf = mean_squared_error(y_valid, rf_y_hat)

r2_linear = r2_score(y_valid, linear_y_hat)
r2_ridge = r2_score(y_valid, ridge_y_hat)
r2_lasso = r2_score(y_valid, lasso_y_hat) 
r2_enet = r2_score(y_valid, enet_y_hat)
r2_rf = r2_score(y_valid, rf_y_hat)

print('OLS\tMSE: {}, R^2: {}'.format(mse_linear, r2_linear))
print('Ridge\tMSE: {}, R^2: {}'.format(mse_ridge, r2_ridge))
print('LASSO\tMSE: {}, R^2: {}'.format(mse_lasso, r2_lasso))
print('Elastic-Net\tMSE: {}, R^2: {}'.format(mse_enet, r2_enet)) 
print('Random Forest\tMSE: {}, R^2: {}'.format(mse_rf, r2_rf))
 

## @VC and KQ: It looks like OLS is the best?! -LL 
##Might be worth calculating accuracy scores as well? model.score 

### 5.2 Feature Selection
We have already selected features for the test set in section 3.3.

### 5.3. Predicting on Test Set
The advantage of using both validation set and test set is that we can avoid overestimating the model accuracy. If only a test set is used, then we might overfit the test set when tuning hyperparameters and selecting models. Given the importance of generalizability of models in social science and public policy, it is crucial to carve out a test set that is used only at the last step to gauge the true accuracy.

In [ ]:
## Select best-performing model and predict on the test set.
# accoding to the results on the validation set, OLS has the lowest MSE
y_hat_best = linear_reg.predict(X_test_scaled)
mse_best = mean_squared_error(y_test, y_hat_best)
r2_best = r2_score(y_test, y_hat_best)
print('MSE best (OLS): ', mse_best)
print('R^2 best (OLS): ', r2_best)
print('Hyperparameters used')
print('--------------------')
for param, val in linear_reg.get_params().items():
    print('\t{}: {}'.format(param, val))

# OLS actually performs better on the test set haha -VC

### 5.3. Cross Validation
Tradeoff with the choice of K: When K is small, each model use only a relatively small fraction of the data, so the estimated error has a larger bias. But since the training sets are more diverse, the estimated error will have smaller variance. When K is large, the bias of estimated error becomes smaller since now we use most of the data for training, but the variance will be larger because the different training sets have considerable overlaps.

In [ ]:
#Perform 10-fold cross validation
#The default score for linear reg is R^2, but since we are using MSE for model selection, I changed it to MSE. -VC
mse_scoring = make_scorer(mean_squared_error)
scores = cross_val_score(linear_reg, features_scaled, target, cv =10, scoring=mse_scoring)
print('Cross-validated means MSE:', np.mean(scores))

The MSE calculated from cross validation is slight larger than that of using the train/validation/test split (4.5e-4 vs 4.1e-4).

# 6. Discussion Questions

## 6.1. The bias-variance tradoff and its relevance to ML problems like this one
Bias-variance tradeoff is a general phenomenon in machine learning that models which have smaller training errors on average (low bias) will be more sensitive to small perturbations in training samples (high variance). Ideally, we want to find a predictor that fits data well (low bias) consistently across datasets (low variance). So bias-variance tradeoff serves as a reminder that when using more complicated models to achieve better training accuracy, we might be sacrificing generalizability along the way.

## 6.2. Overfitting, why it matters to ML, and ways to address it
Overfitting is the situation where a machine learning method has high accuracy in the training set, but perfoms poorly in the test set. It usually happens when people use more complicated models in an attempt to boost prediction accuracy, but the additional flexibility is used to fit noises rather than signals. The problem can be addressed by reserving part of the data as the validation set. First fit different models on the training set, and select models based on performance on the validation set. 

## 6.3. Discussion
From the analysis, we identifed three risk factors of diabetes rate: physical inactivity number, obesity rate, and the ratio of African Americans (NOTE: I'm not sure if it's really the meaning of the third variable. Need to check. -VC), in descending order of importance. The order is consistent across all five models that we used, suggesting that the result is robust to the machine learning methods chosen. All three features are positively correlated with diabetes rate.

We recommend prioritizing the counties which (a) already have high diabetes rate or (b) do not currently have high diabetes rate but are so predicted by our OLS regression model. For group (a), the high diabetes rate indicates that there might be systematic factors in the counties that increase the likelihood of developing diabetes, therefore conducting the pilot program there could potentially reduce the future diabetes rate. For group (b), even though the counties do not have high diabetes rate yet, according to our model it is likely that more residents will develop diabetes in the near future. A successful prevention program could prevent that from happening. The results are shown in the following tables:

In [ ]:
# Top 10 counties with highest diabetes rate
df.sort_values('Diabetes_Number', ascending=False)[['County', 'State', 'Diabetes_Number']].head(10)

In [ ]:
# Top 10 counties with highest predicted diabetes rate
topn = 10
y_hat_all = linear_reg.predict(features_scaled).tolist()
index_ranked = list(range(len(y_hat_all)))
# sort county index by predicted y_hat
index_yhat = sorted(zip(index_ranked, y_hat_all), key=lambda x: x[1], reverse=True)
index_ranked = [p[0] for p in index_yhat]
yhat_ranked = np.array([p[1] for p in index_yhat])
# get county by index
df_highest_yhat = df.loc[index_ranked[:topn]][['County', 'State', 'Diabetes_Number']]
df_highest_yhat['Predicted Diabetes Number'] = yhat_ranked[:topn]
df_highest_yhat

We feel pretty confident (or do we? haha -VC) about deploying this model in real-world applications. It is not only because the model performs pretty well on the test set which is untouched throughout the training process, but also because the relative importance of features are consistent across the five models we have tried. More generally, the approach taken in this project allows us to systematically test the suitability of different machine learning models. If the selected model performs well on the test set, we can be assured about its generalizability outside the training data. Even if all of the models fail to provide satisfactory results, we can safely conclude that the problem at hand is beyond the reach of the models we have tried. Overall, we feel comfortable about applying machine learning methods to our fields of study.